# Сортировка по совпадению документов

Было принято решение обрабатывать файлы по отдельности, так как не во всех файлах существуют документы хоть в каком-нибудь виде.

## 1. BoardingData.csv

In [6]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('BoardingData.csv', sep=';')

# Удаляем пробелы в названиях столбцов
df.columns = df.columns.str.strip()

filtered_df = (df.groupby('PassengerDocument')
               .filter(lambda x: x['PassengerFirstName'].nunique() > 1 or x['PassengerLastName'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates(subset=['PassengerLastName'])

if final_df.empty:
    print("Совпадений не найдено")
else:
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersCSV.csv', index=False) 
    print(final_df)
    

       PassengerFirstName PassengerSecondName PassengerLastName PassengerSex  \
83246              GORDEI                  M.            GLEBOV         Male   
150151              RAMIL         ALBERTOVICH             BUROV         Male   

       PassengerBirthDate PassengerDocument    BookingCode      TicketNumber  \
83246          12/31/1982       8248 013778  Not presented  5858732412927507   
150151         08/24/1995       8248 013778  Not presented  7230123462656045   

           Baggage  FlightDate FlightTime FlightNumber CodeShare Destination  
83246   Registered  2017-03-09      07:15       SU1276       Own       Kazan  
150151  Registered  2017-01-03      11:15       SU1273       Own      Moscow  


## 2. JSON. 
Сначала перевод в CSV для удобства. Файл - FrequentFlyerForum-Profiles.json

In [5]:
import pandas as pd
import json
with open('./data/Airlines/FrequentFlyerForum-Profiles.json', 'r') as f:
    data = json.load(f)

# Преобразование данных JSON в pandas DataFrame
profiles = data["Forum Profiles"]

parsed_data = []
for profile in profiles:
    first_name = profile["Real Name"]["First Name"]
    last_name = profile["Real Name"]["Last Name"]
    sex = profile["Sex"]
    document = profile["Travel Documents"][0]["Passports"]
    
    for flight in profile["Registered Flights"]:
        if profile.get("Loyality Programm"):
            loyality_program = profile["Loyality Programm"][0]
            loy_prgrm_status = loyality_program.get("Status", None)
            loy_prgrm_code = loyality_program.get("programm", None)
            loy_prgrm_number = loyality_program.get("Number", None)
        else:
            loy_prgrm_status = loy_prgrm_code = loy_prgrm_number = None
        
        parsed_data.append({
            "First Name": first_name,
            "Last Name": last_name,
            "Sex": sex,
            "Document": document,
            "Flight Date": flight["Date"],
            "Flight Number": flight["Flight"],
            "Share Code": flight["Codeshare"],
            "ArrivalCity": flight["Arrival"]["City"],
            "ArrivalAirport": flight["Arrival"]["Airport"], 
            "DepartureCity": flight["Departure"]["City"],   
            "DepartureAirport": flight["Departure"]["Airport"],
            "NickName": profile.get("NickName", None), 
            "LoyPrgrmStatus": loy_prgrm_status,
            "LoyPrgrmCode": loy_prgrm_code,
            "LoyPrgrmNumber": loy_prgrm_number
        })

# Преобразование списка в pandas DataFrame
df_json = pd.DataFrame(parsed_data)
df_json.to_csv('FrequentFlyerForum-Profiles-json.csv', index=False)

In [10]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('FrequentFlyerForum-Profiles-json.csv', sep=',')

# Удаляем пробелы в названиях столбцов
df.columns = df.columns.str.strip()

filtered_df = (df.groupby('Document')
               .filter(lambda x: x['First Name'].nunique() > 1 or x['Last Name'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersJSON.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

Совпадений не найдено


## 3. Sirena-export-fixed.tab. 
Схема та же, что и с JSON.

In [14]:
colspecs = [
    (0, 60),   # PaxName
    (60, 72),  # PaxBirthDate
    (72, 84),  # DepartDate
    (84, 96),  # DepartTime
    (96, 108),  # ArrivalDate
    (108, 120),  # ArrivalTime
    (120, 126),  # Flight
    (126, 132),  # CodeSh
    (132, 138),  # From
    (138, 144),  # Dest
    (144, 150),  # Code
    (150, 168),  # e-Ticket
    (168, 180), # TravelDoc
    (180, 186),# Seat
    (186, 192),# Meal
    (192, 198),# TrvCls
    (198, 216),# Fare  Baggage
    (216, 240),# PaxAdditionalInfo
    (240, 246),# Unnamed: 18
    (246, 276), # Unnamed: 19
    (276, 336) # AgentInfo
]

df_tab = pd.read_fwf('./data/Airlines/Sirena-export-fixed.tab', colspecs=colspecs, dtype={'Unnamed: 19': 'str', 'Unnamed: 18': 'str'})

# Удаляем 'FF#' в столбце Unnamed: 18, оставляем только программный код (например, 'DT')
df_tab['Unnamed: 18'] = df_tab['Unnamed: 18'].str.replace('FF#', '', regex=False)

# Шаг 4: Разделение PaxName на три части (Last Name, First Name, Second Name)
#df_tab[['Last Name', 'First Name', 'Second Name']] = df_tab['PaxName'].str.split(expand=True, n=2)

# Шаг 5: Переименование и объединение столбцов из Sirena с основным датасетом
df_tab = df_tab.rename(columns={
    'PaxBirthDate': 'Birth Date',
    'Flight': 'Flight Number',
    'CodeSh': 'Share Code',
    'From': 'DepartureAirport',
    'Dest': 'ArrivalAirport',
    'Code': 'BookingCode',
    'e-Ticket': 'TicketNumber',
    'TravelDoc': 'Document',
    'Fare  Baggage': 'Baggage',
    'DepartDate': 'Flight Date',
    'DepartTime': 'FlightTime',
    'Unnamed: 19': 'LoyPrgrmNumber',
    'Unnamed: 18': 'LoyPrgrmCode'
})

df_tab.to_csv('Sirena-export-fixed-tab.csv', index=False)

In [15]:
df_tab.head(10)

,PaxName,Birth Date,Flight Date,FlightTime,ArrivalDate,ArrivalTime,Flight Number,Share Code,DepartureAirport,ArrivalAirport,...,TicketNumber,Document,Seat,Meal,TrvCls,Baggage,PaxAdditionalInfo,LoyPrgrmCode,LoyPrgrmNumber,AgentInfo
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,...,7360415302044672,9375 053270,NaN,NaN,J,JGRPGN0PC,S,SU,38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,...,7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,9,FB,284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,...,5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,F,NaN,NaN,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,5274206497242737,98 6865148,NaN,NaN,J,JFLXLS0PC,#,FB,884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,SU,183142068,OZON.travel
5,ГЛУШКОВ КОНСТАНТИН ИЛЬИЧ,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,5874178506968181,4788 422492,NaN,NaN,Y,YGRPPN0PC,8,FB,553284496,KupiBilet
6,КАПУСТИН АРТЁМ ЭДУАРДОВИЧ,1982-10-24,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,7467749130398378,0058 142289,NaN,VLML,Y,YFLXNM2PC,F,NaN,NaN,KupiBilet
7,ЕРШОВА ЛЮБОВЬ ЗАХАРОВНА,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,2183161939566868,0776 380126,NaN,NaN,Y,YGRPYX0PC,#,NaN,NaN,Aeroflot
8,ТИТОВА ЗАРИНА ЭМИЛЬЕВНА,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,SVO,...,5954073786122008,53 7554162,NaN,NaN,Y,YSTNXJ1PC,8,FB,933538031,OZON.travel
9,БАРСУКОВ САМИР ГОРДЕЕВИЧ,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,SVO,...,2264717979478322,0078 271703,NaN,NaN,Y,YGRPPP1PC,F,NaN,NaN,Aeroflot


In [18]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('Sirena-export-fixed-tab.csv', sep=',')

# Удаляем пробелы в названиях столбцов
#df.columns = df.columns.str.strip()

filtered_df = (df.groupby('Document')
               .filter(lambda x: x['PaxName'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersTAB.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

                         PaxName  Birth Date Flight Date FlightTime  \
93152    ГЛЕБОВ ГОРДЕЙ МАКАРЕВИЧ  1982-12-31  2017-01-16      01:45   
150692  БУРОВ РАМИЛЬ АЛЬБЕРТОВИЧ  1995-08-24  2017-01-03      11:15   

       ArrivalDate ArrivalTime Flight Number Share Code DepartureAirport  \
93152   2017-01-16       04:35        SU1057         NO              MCX   
150692  2017-01-03       13:35        SU1273         NO              KRR   

       ArrivalAirport  ...      TicketNumber     Document Seat  Meal TrvCls  \
93152             SVO  ...  2082229451418171  8248 013778  NaN  KSML      Y   
150692            SVO  ...  7230123462656045  8248 013778  NaN   NaN      A   

       Baggage PaxAdditionalInfo LoyPrgrmCode LoyPrgrmNumber  AgentInfo  
93152   YGRPFD                 F          NaN            NaN    trip.ru  
150692  ASTNLM               NaN          NaN            NaN        NaN  

[2 rows x 21 columns]


## 4. XLSX. 
В отдельном ноутбуке было сведение и перевод в CSV формат всех таблиц. В этом файле нет колонки с документами, поэтому происходит сравнение на билет. (ну пусть будет) (пока что получается хрень)

In [4]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('xlsx_data.csv', sep=',')

# Удаляем пробелы в названиях столбцов
#df.columns = df.columns.str.strip()

filtered_df = (df.groupby('TicketNumber')
               .filter(lambda x: x['Full Name'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates(subset=['Full Name'])

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersXLSX.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

Совпадений не найдено


## 5. XML.

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
tree = ET.parse('./data/Airlines/PointzAggregator-AirlinesData.xml')
root = tree.getroot()

# Создаем список для хранения данных
data = []

# Проходим по каждому пользователю
for user in root.findall('user'):
    uid = user.get('uid')  # ID пользователя
    first_name = user.find('name').get('first')  # Имя пользователя
    last_name = user.find('name').get('last')  # Фамилия пользователя
    
    # Проходим по каждой карточке
    for card in user.findall('cards/card'):
        card_number = card.get('number')  # Номер карты
        bonus_program = card.find('bonusprogramm').text  # Бонусная программа
        
        # Проходим по каждой активности (рейсу)
        for activity in card.findall('activities/activity'):
            flight_code = activity.find('Code').text  # Код рейса
            flight_date = activity.find('Date').text  # Дата рейса
            departure = activity.find('Departure').text  # Место вылета
            arrival = activity.find('Arrival').text  # Место прилета
            fare = activity.find('Fare').text  # Тип тарифа
            
            # Добавляем все данные в список
            data.append({
                'User uid': uid,
                'First Name': first_name,
                'Last Name': last_name,
                'CardNumber': card_number,
                'BonusProgram': bonus_program,
                'Flight Number': flight_code,
                'Flight Date': flight_date,
                'DepartureAirport': departure,
                'ArrivalAirport': arrival
            })

# Преобразуем список в DataFrame
df_xml = pd.DataFrame(data)
df_xml.to_csv('PointzAggregator-AirlinesData-xml.csv', index=False)

In [3]:
df_xml.head(10)

,User uid,First Name,Last Name,CardNumber,BonusProgram,Flight Number,Flight Date,DepartureAirport,ArrivalAirport
0,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,KE827,2017-08-06,rea,SZX
1,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,MU9706,2017-10-26,PEK,BSD
2,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL5058,2017-09-11,CHA,ATL
3,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,KE1,2017-04-01,rea,HNL
4,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL837,2017-09-13,ATL,HNL
5,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,SU1523,2017-02-12,NUX,SVO
6,138879468,DANIL,VAVILOV,DT 980250352,Delta SkyMiles,AF4445,2017-11-11,AJA,ORY
7,138879468,DANIL,VAVILOV,DT 980250352,Delta SkyMiles,AZ7545,2017-03-07,FCO,PRG
8,138879468,DANIL,VAVILOV,DT 980250352,Delta SkyMiles,AF7331,2017-09-17,MLH,ORY
9,138879468,DANIL,VAVILOV,DT 980250352,Delta SkyMiles,AF4102,2017-10-07,ORY,AJA


In [2]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('PointzAggregator-AirlinesData-xml.csv', sep=',')

# Удаляем пробелы в названиях столбцов
#df.columns = df.columns.str.strip()

filtered_df = (df.groupby('CardNumber')
               .filter(lambda x: x['First Name'].nunique() > 1 or x['Last Name'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates(subset=['Last Name', 'First Name'])

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersXML.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

         User uid First Name    Last Name    CardNumber BonusProgram  \
130045  146589094      ROMAN  MASLENNIKOV  FB 139888197  Flying Blue   
312850  866799462     OLESYA     GOLOVINA  FB 139888197  Flying Blue   

       Flight Number Flight Date DepartureAirport ArrivalAirport  
130045        MU5529  2017-08-17              SHA            DQA  
312850        ME2231  2017-07-29              BEY            FCO  


# Построение карты
После выявления наиболее и наименее посещаемых городов, которые были даны списком, нужно было разместить их на карту.

## 1. Находим координаты.

In [ ]:
import requests

def get_coordinates_opencage(city, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={city}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']
            return (location['lng'], location['lat'])
        else:
            print(f"Координаты для {city} не найдены.")
            return None
    else:
        print(f"Ошибка при получении координат для {city}: {response.status_code}")
        return None

# Ваш API-ключ OpenCage
api_key = '378915dab2e24376b44d14d2a49bb97d'  # Замените на ваш API-ключ

# Список городов
cities = [
    "rio cuarto",
    "providenciales",
    "sintang",
    "cozumel",
    "kingston",
    "santa rosa",
    "regina",
    "cartagena",
    "medellin",
    "roatan",
    "moscow",
    "paris",
    "atlanta ga",
    "new york ny",
    "amsterdam",
    "shanghai",
    "rome",
    "seoul",
    "detroit mi",
    "minneapolis"
]

# Получение координат для каждого города
coordinates = {}
for city in cities:
    coords = get_coordinates_opencage(city, api_key)
    if coords:
        coordinates[city] = coords

# Вывод результатов
for city, coord in coordinates.items():
    print(f"{city}, {coord[0]}, {coord[1]}")

## 2. Создание карты и сохранение в файл

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

# Загрузка данных из CSV файлов
most_visited = pd.read_csv('./data/cities_most.csv')
least_visited = pd.read_csv('./data/cities_list.csv')

# Создание карты
plt.figure(figsize=(15, 10))  # Увеличение размера изображения
map = Basemap(projection='merc', lat_0=0, lon_0=0, resolution='i')  # Используем 'i' для более высокой детализации

# Нанесение сетки на карту
map.drawmapboundary(fill_color='#46bcec')
map.fillcontinents(color='#f2f2f2', lake_color='#46bcec')
map.drawcoastlines()
map.drawcountries()   # Добавление границ стран для большей точности

# Добавление точек для наиболее посещаемых городов (синий цвет)
lon_most = most_visited['lon'].values
lat_most = most_visited['lat'].values
x_most, y_most = map(lon_most, lat_most)
map.scatter(x_most, y_most, 30, marker='o', color='blue', zorder=5)

# Добавление точек для наименее посещаемых городов (красный цвет)
lon_least = least_visited['lon'].values
lat_least = least_visited['lat'].values
x_least, y_least = map(lon_least, lat_least)
map.scatter(x_least, y_least, 30, marker='o', color='red', zorder=5)

# Убираем подписи координат
map.drawparallels([])  # Убираем параллели
map.drawmeridians([])  # Убираем меридианы

# Добавление легенды
plt.legend(loc='upper right')

# Сохранение карты в файл
plt.savefig('combined_cities_map.png', bbox_inches='tight', dpi=1200)  # Увеличение разрешения изображения

print("Карта успешно создана и сохранена в файл 'combined_cities_map.png'.")